In [1]:
import tensorflow as tf 
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt 

import tensorflow_datasets as tfds

In [2]:
dataset, info = tfds.load('glue/sst2', with_info=True)

ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html